In [17]:
import pandas as pd 
from sklearn.model_selection import train_test_split

In [26]:
data = pd.read_csv("data/ner_dataset.csv",sep=",",encoding="latin1").fillna(method='ffill')

In [27]:
data = data.drop(['POS'], axis=1)

In [28]:
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [29]:
getter = SentenceGetter(data)

In [30]:
sentences = getter.sentences

In [31]:
len(sentences)

47959

In [32]:
train, test = train_test_split(sentences, test_size=0.2, random_state=42, shuffle=True)

In [33]:
train, val = train_test_split(train, test_size=0.25, random_state=42, shuffle=True)

In [38]:
def to_df(data):
    n_sent = 1
    words = []
    tags = []
    nums = []
    for i in range(len(data)):
        for j in range(len(data[i])):
            nums.append("Sentence: {}".format(n_sent))
            words.append(data[i][j][0])
            tags.append(data[i][j][1])
        n_sent = n_sent + 1
    df = pd.DataFrame({'Sentence #':nums, 'Word':words, 'Tag':tags})
    return df

In [44]:
train_df = to_df(train)
val_df = to_df(val)
test_df = to_df(test)

In [45]:
train_df.to_csv("data/train.csv")
val_df.to_csv("data/val.csv")
test_df.to_csv("data/test.csv")